In [ ]:
#!/usr/bin/env python
# -*-coding:utf-8 -*-
'''
@File    :   weather.ipynb
@Time    :   2024/10/30 17:44:38
@Author  :   Guillermo Martin
@Version :   1.0
@Personal email : gfmg1992@hotmail.com
@Student email: G00438885@atu.ie
@License :   (C)Copyright 2023, Guillermo Martin
@Desc    :   Notebook describing how each of the task for the Computer infrastructure course were done
'''


# Task 1: Create a directory structure
Within VS code. I create a new terminal, and select Git Bash instead of powershell (default). I create a new folder within my `CI_gfmg` repository called `data` by doing in Bash: 

> mkdir data

I then change my current directory to the new `data` folder by doing in bash: 

> cd data

Within this folder, I now create two new folders by: 
> mkdir timestamps

> mkdir weather

# Task 2: Timestamps
I first change my current directory to the recently created timestamps folder by: 

> cd ./data/timestamps 

I now append the current date to a new `.txt` file by doing:
> date >> now.txt

I repeated the above command 10 times, and then I read the `now.txt` file by doing:

>cat now.txt

For some reason, within VS code using Bash (not codespace) there are some commands noy available. For instance `more` or `man`. 

# Task 3: Formatting Timestamps
Using the terminal GitBash within VS code, I typed the following: 

>date +'%Y%m%d_%H%M%S' >> formatted.txt 

I have also tested how to use it within GitHub CodeSpace. 

# Task 4: Create Timestamped Files
To create a `.txt` file with the current date and time in the format `YYYmmdd_HHMMSS` I used in bash the following: 

>touch `date +"%Y%m%d_%H%M%S.txt"`

# Task 5: Download Today's Weather Data
I already mentioned that, working in bash from VS code, there was a couple of bash functions that were not available including `more` or `man`. I found out that I can run Codespaces from my desktop using a VS Code extension. I found the information to do so here: https://code.visualstudio.com/docs/sourcecontrol/intro-to-git

I could also run it from GitHub directly, but I prefer to do it from my desktop. Within CodeSpace I now run: 

> wget -O weather.json https://prodapi.metweb.ie/observations/athenry/today 

The `-O` options allows to save the data within the URL with the name `weather.json`

# Task 6: Timestamp the Data

For this task, we literally combine the commands that we developed in Task 5 and Task 4: 

wget -O `date +"%Y%m%d_%H%M%S.json"` https://prodapi.metweb.ie/observations/athenry/today 

# Task 7: Write the Script